In [1]:
import qiskit

In [2]:
from qiskit import QuantumCircuit

In [3]:
import math

In [4]:
def half_adder(qc):

    #mesuare all qbits 0, 1 and 2
    qc.measure(0,0)
    qc.measure(1,1)
    qc.measure(2,2)

    #apply CNOT on qbits 1 and 2 with qbit 3 as control
    qc.cx(1,3)
    qc.cx(2,3)

    #apply CNOT on qbits 0 and 3 with qbit 4 as control
    qc.cx(0,4)
    qc.cx(3,4)

    #mesuare qbit 4 with contain the result of the sum of the qbits 0, 1 and 2
    qc.measure(4,3)

In [5]:
def carry_out(qc):
    #apply CCNOT on qbits 1 and 2 with qbit 5 as control
    qc.ccx(1,2,5)
    #apply CCNOT on qbits 0 and 3 with qbit 6 as control
    qc.ccx(0,3,6)

    #invert qbits 5 and 6
    qc.x(5)
    qc.x(6)

    #apply CcNOT on qbits 5 and 6 with qbit 7 as control
    qc.ccx(5,6,7)

    #reversible inversion of qbit 7
    qc.rx(math.pi,7)

    #measure qbit 7 with contain the carry out of the sum of the qbits 0, 1 and 2
    qc.measure(7,4)

In [6]:
#create quantum circuit with 7 qbits and 4 classical bits
qc = QuantumCircuit(8,5, name="full_adder")

#set to 1 all qbits
qc.x(0)
qc.x(1)
qc.x(2)

half_adder(qc)
carry_out(qc)

https://docs.quantum.ibm.com/guides/primitives-examples

In [7]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
 
n_qubits = 8
 
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=n_qubits)
 
circuit = qc
observable = SparsePauliOp("Z" * n_qubits)
 
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
 
estimator = Estimator(backend)
job = estimator.run([(isa_circuit, isa_observable)])
result = job.result()
 
print(f" > Expectation value: {result[0].data.evs}")
print(f" > Metadata: {result[0].metadata}")
#print(result[0].data.c.get_counts())

 > Expectation value: -0.0009765625
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'num_randomizations': 32}


AttributeError: 'DataBin' object has no attribute 'c'